## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-19-14-52-35 +0000,48hills,‘Poetry for Men’,https://48hills.org/2025/11/poetry-for-men/
1,2025-11-19-14-51-59 +0000,bbc,"Labour's Budget is already unravelling, Badeno...",https://www.bbc.com/news/articles/c5ypy5v21l7o...
2,2025-11-19-14-49-00 +0000,wsj,"Europe Aimed to Set Standards for Tech Rules, ...",https://www.wsj.com/world/europe/europe-aimed-...
3,2025-11-19-14-47-00 +0000,wsj,Target to Invest Billions to Upgrade Stores as...,https://www.wsj.com/business/retail/target-tgt...
4,2025-11-19-14-46-51 +0000,nypost,"Volcano erupts, sending ash over 6,500 feet in...",https://nypost.com/2025/11/19/world-news/volca...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,52
39,epstein,30
48,saudi,19
117,new,16
264,house,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
236,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...,144
255,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,142
191,2025-11-19-01-33-54 +0000,nypost,Trump announces Saudi Arabia will be designate...,https://nypost.com/2025/11/18/us-news/trump-an...,126
299,2025-11-18-19-44-18 +0000,nypost,Congress votes to send Epstein files release b...,https://nypost.com/2025/11/18/us-news/house-vo...,121
203,2025-11-19-00-58-13 +0000,cbc,"Once shunned, Saudi crown prince is Trump’s g...",https://www.cbc.ca/news/world/saudi-arabia-cro...,115


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
236,144,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
191,74,2025-11-19-01-33-54 +0000,nypost,Trump announces Saudi Arabia will be designate...,https://nypost.com/2025/11/18/us-news/trump-an...
255,60,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
266,55,2025-11-18-21-46-47 +0000,latimes,"'Played with fire, got burned': GOP control of...",https://www.latimes.com/politics/story/2025-11...
29,55,2025-11-19-14-04-00 +0000,wsj,EU Proposes Multibillion-Euro Plan to Boost Mi...,https://www.wsj.com/world/europe/eu-proposes-m...
73,40,2025-11-19-11-47-57 +0000,nypost,Rep. Clay Higgins explains why he was the only...,https://nypost.com/2025/11/19/us-news/louisian...
59,35,2025-11-19-12-53-53 +0000,nypost,Larry Summers resigns from OpenAI board follow...,https://nypost.com/2025/11/19/business/larry-s...
321,33,2025-11-18-18-36-00 +0000,nypost,Dow plunges nearly 500 points on fears of AI b...,https://nypost.com/2025/11/18/business/dow-plu...
57,33,2025-11-19-13-00-38 +0000,nypost,NYPD Commissioner Jessica Tisch accepts Mayor-...,https://nypost.com/2025/11/19/us-news/nypd-com...
252,33,2025-11-18-22-19-58 +0000,bbc,Lebanon says Israeli strike killed 13 people n...,https://www.bbc.com/news/articles/c0l73j781y9o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
